# بناء نموذج انحدار: نماذج الانحدار الخطي والمتعدد


## الانحدار الخطي والانحدار متعدد الحدود لتسعير القرع - الدرس الثالث
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>رسم توضيحي من تصميم داساني ماديبالي</figcaption>

#### المقدمة

حتى الآن، قمت باستكشاف مفهوم الانحدار باستخدام بيانات عينة مأخوذة من مجموعة بيانات تسعير القرع التي سنستخدمها طوال هذا الدرس. كما قمت بتصورها باستخدام `ggplot2`. 💪

الآن أنت مستعد للتعمق أكثر في الانحدار لتعلم الآلة. في هذا الدرس، ستتعلم المزيد عن نوعين من الانحدار: *الانحدار الخطي الأساسي* و *الانحدار متعدد الحدود*، بالإضافة إلى بعض الرياضيات التي تقوم عليها هذه التقنيات.

> خلال هذا المنهج، نفترض معرفة رياضية بسيطة، ونسعى لجعلها سهلة الفهم للطلاب القادمين من مجالات أخرى، لذا انتبه للملاحظات 🧮، والرسوم التوضيحية، وأدوات التعلم الأخرى التي تساعد على الفهم.

#### التحضير

كتذكير، تقوم بتحميل هذه البيانات لطرح أسئلة عليها.

- متى يكون أفضل وقت لشراء القرع؟

- ما السعر الذي يمكن أن أتوقعه لصندوق من القرع الصغير؟

- هل يجب أن أشتريه في سلال نصف بوشل أم في صناديق 1 1/9 بوشل؟ دعنا نستمر في استكشاف هذه البيانات.

في الدرس السابق، قمت بإنشاء `tibble` (إعادة تصور حديثة لإطار البيانات) وملأته بجزء من مجموعة البيانات الأصلية، مع توحيد الأسعار حسب البوشل. ومع ذلك، من خلال القيام بذلك، تمكنت فقط من جمع حوالي 400 نقطة بيانات وللأشهر الخريفية فقط. ربما يمكننا الحصول على مزيد من التفاصيل حول طبيعة البيانات من خلال تنظيفها أكثر؟ سنرى... 🕵️‍♀️

لإتمام هذه المهمة، سنحتاج إلى الحزم التالية:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) هو [مجموعة من حزم R](https://www.tidyverse.org/packages) مصممة لجعل علم البيانات أسرع وأسهل وأكثر متعة!

- `tidymodels`: إطار عمل [tidymodels](https://www.tidymodels.org/) هو [مجموعة من الحزم](https://www.tidymodels.org/packages/) للنمذجة وتعلم الآلة.

- `janitor`: حزمة [janitor](https://github.com/sfirke/janitor) توفر أدوات بسيطة لفحص وتنظيف البيانات غير المنظمة.

- `corrplot`: حزمة [corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) تقدم أداة استكشافية بصرية لمصفوفة الارتباط تدعم إعادة ترتيب المتغيرات تلقائيًا للمساعدة في اكتشاف الأنماط المخفية بين المتغيرات.

يمكنك تثبيتها باستخدام:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

البرنامج النصي أدناه يتحقق مما إذا كانت الحزم المطلوبة لإكمال هذا الدرس مثبتة لديك، ويقوم بتثبيتها إذا كانت مفقودة.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

سنقوم لاحقًا بتحميل هذه الحزم الرائعة وجعلها متاحة في جلسة R الحالية لدينا. (هذا للتوضيح فقط، `pacman::p_load()` قام بذلك بالفعل نيابةً عنك)

## 1. خط الانحدار الخطي

كما تعلمت في الدرس الأول، الهدف من تمرين الانحدار الخطي هو القدرة على رسم *خط* *أفضل تطابق* لـ:

-   **إظهار العلاقات بين المتغيرات**. إظهار العلاقة بين المتغيرات.

-   **إجراء التنبؤات**. تقديم تنبؤات دقيقة حول مكان وقوع نقطة بيانات جديدة بالنسبة لهذا الخط.

لرسم هذا النوع من الخطوط، نستخدم تقنية إحصائية تُسمى **الانحدار بطريقة المربعات الصغرى**. مصطلح `المربعات الصغرى` يعني أن جميع نقاط البيانات المحيطة بخط الانحدار يتم تربيعها ثم جمعها. من الناحية المثالية، يكون المجموع النهائي صغيرًا قدر الإمكان، لأننا نريد عددًا منخفضًا من الأخطاء، أو `المربعات الصغرى`. وبالتالي، فإن خط أفضل تطابق هو الخط الذي يعطينا أقل قيمة لمجموع الأخطاء المربعة - ومن هنا جاء اسم *الانحدار بطريقة المربعات الصغرى*.

نقوم بذلك لأننا نريد نمذجة خط يكون لديه أقل مسافة تراكمية من جميع نقاط البيانات لدينا. كما نقوم بتربيع القيم قبل جمعها لأننا نهتم بحجمها وليس اتجاهها.

> **🧮 أرني الرياضيات**
>
> يمكن التعبير عن هذا الخط، المسمى *خط أفضل تطابق* بواسطة [معادلة](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` هو '`المتغير التوضيحي` أو `المتنبئ`'. `Y` هو '`المتغير التابع` أو `النتيجة`'. ميل الخط هو `b` و`a` هو نقطة تقاطع المحور y، والتي تشير إلى قيمة `Y` عندما يكون `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "الميل = $y/x$")
    رسم توضيحي بواسطة Jen Looper
>
> أولاً، احسب الميل `b`.
>
> بمعنى آخر، وبالإشارة إلى سؤال البيانات الأصلية الخاصة بالقرع: "توقع سعر القرع لكل بوشل حسب الشهر"، سيكون `X` هو السعر و`Y` هو شهر البيع.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.ar.png)
    رسم توضيحي بواسطة Jen Looper
> 
> احسب قيمة Y. إذا كنت تدفع حوالي \$4، فلا بد أن يكون شهر أبريل!
>
> يجب أن تُظهر الرياضيات التي تحسب الخط ميل الخط، والذي يعتمد أيضًا على نقطة التقاطع، أو مكان وجود `Y` عندما يكون `X = 0`.
>
> يمكنك ملاحظة طريقة الحساب لهذه القيم على موقع [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). كما يمكنك زيارة [آلة حاسبة المربعات الصغرى](https://www.mathsisfun.com/data/least-squares-calculator.html) لمشاهدة كيف تؤثر قيم الأرقام على الخط.

ليس مخيفًا جدًا، صحيح؟ 🤓

#### الارتباط

مصطلح آخر يجب فهمه هو **معامل الارتباط** بين المتغيرين X وY المعطى. باستخدام مخطط التبعثر، يمكنك بسرعة تصور هذا المعامل. المخطط الذي يحتوي على نقاط بيانات متناثرة في خط مرتب لديه ارتباط عالي، ولكن المخطط الذي يحتوي على نقاط بيانات متناثرة في كل مكان بين X وY لديه ارتباط منخفض.

نموذج الانحدار الخطي الجيد سيكون النموذج الذي لديه معامل ارتباط عالي (أقرب إلى 1 من 0) باستخدام طريقة الانحدار بطريقة المربعات الصغرى مع خط الانحدار.


## **2. رقصة مع البيانات: إنشاء إطار بيانات سيتم استخدامه للنمذجة**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>


قم بتحميل المكتبات والبيانات المطلوبة. قم بتحويل البيانات إلى إطار بيانات يحتوي على جزء من البيانات:

-   احصل فقط على القرع الذي يتم تسعيره بالوحدة (البوشل).

-   قم بتحويل التاريخ إلى شهر.

-   احسب السعر ليكون متوسط الأسعار العالية والمنخفضة.

-   قم بتحويل السعر ليعكس التسعير حسب كمية البوشل.

> قمنا بتغطية هذه الخطوات في [الدرس السابق](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

في روح المغامرة الخالصة، دعونا نستكشف [`حزمة janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) التي توفر وظائف بسيطة لفحص وتنظيف البيانات غير النظيفة. على سبيل المثال، دعونا نلقي نظرة على أسماء الأعمدة لبياناتنا:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 يمكننا أن نفعل أفضل. دعونا نجعل أسماء الأعمدة هذه `friendR` عن طريق تحويلها إلى [snake_case](https://en.wikipedia.org/wiki/Snake_case) باستخدام `janitor::clean_names`. لمعرفة المزيد عن هذه الدالة: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

الكثير من الترتيب 🧹! الآن، رقصة مع البيانات باستخدام `dplyr` كما في الدرس السابق! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

عمل رائع! 👌 لديك الآن مجموعة بيانات نظيفة ومرتبة يمكنك بناء نموذج الانحدار الجديد عليها!

هل تمانع في رسم مخطط مبعثر؟


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


مخطط التبعثر يذكرنا بأن البيانات المتوفرة لدينا للشهور تمتد فقط من أغسطس إلى ديسمبر. من المحتمل أننا بحاجة إلى المزيد من البيانات لنتمكن من استخلاص استنتاجات بطريقة خطية.

دعونا نلقي نظرة مرة أخرى على بيانات النمذجة الخاصة بنا:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

ماذا لو أردنا التنبؤ بـ `السعر` لقرع بناءً على أعمدة `المدينة` أو `الحزمة` التي تكون من نوع نصي؟ أو بشكل أبسط، كيف يمكننا العثور على الارتباط (الذي يتطلب أن تكون مدخلاته رقمية) بين، على سبيل المثال، `الحزمة` و `السعر`؟ 🤷🤷

نماذج التعلم الآلي تعمل بشكل أفضل مع الميزات الرقمية بدلاً من القيم النصية، لذا عادةً ما تحتاج إلى تحويل الميزات الفئوية إلى تمثيلات رقمية.

هذا يعني أننا بحاجة إلى إيجاد طريقة لإعادة صياغة المتنبئات لدينا لجعلها أسهل للاستخدام بشكل فعال من قبل النموذج، وهي عملية تُعرف بـ `هندسة الميزات`.


## 3. تجهيز البيانات للنمذجة باستخدام الوصفات 👩‍🍳👨‍🍳

الأنشطة التي تعيد صياغة قيم المتنبئات لجعلها أسهل للاستخدام بشكل فعال من قبل النموذج تُعرف بـ `هندسة الميزات`.

تتطلب النماذج المختلفة متطلبات معالجة مسبقة مختلفة. على سبيل المثال، تتطلب طريقة المربعات الصغرى `ترميز المتغيرات الفئوية` مثل الشهر، النوع، واسم المدينة. يتضمن هذا ببساطة `ترجمة` عمود يحتوي على `قيم فئوية` إلى واحد أو أكثر من `الأعمدة الرقمية` التي تحل محل الأصل.

على سبيل المثال، لنفترض أن بياناتك تحتوي على الميزة الفئوية التالية:

|  المدينة   |
|:----------:|
| دنفر       |
| نيروبي     |
| طوكيو      |

يمكنك تطبيق *الترميز الترتيبي* لاستبدال قيمة عددية فريدة لكل فئة، مثل هذا:

| المدينة |
|:-------:|
|   0     |
|   1     |
|   2     |

وهذا ما سنفعله مع بياناتنا!

في هذا القسم، سنستكشف حزمة رائعة أخرى من Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - والتي تم تصميمها لمساعدتك في تجهيز بياناتك **قبل** تدريب النموذج. في جوهرها، الوصفة هي كائن يحدد الخطوات التي يجب تطبيقها على مجموعة البيانات لجعلها جاهزة للنمذجة.

الآن، دعونا ننشئ وصفة تُجهز بياناتنا للنمذجة عن طريق استبدال قيمة عددية فريدة لجميع الملاحظات في أعمدة المتنبئات:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

رائع! 👏 لقد أنشأنا للتو أول وصفة تحدد النتيجة (السعر) والمتنبئين المقابلين لها، وتأكدنا أن جميع أعمدة المتنبئين يتم ترميزها إلى مجموعة من الأعداد الصحيحة 🙌! دعونا نوضح ذلك بسرعة:

-   استدعاء `recipe()` مع صيغة يوضح للوصفة *الأدوار* الخاصة بالمتغيرات باستخدام بيانات `new_pumpkins` كمرجع. على سبيل المثال، تم تعيين عمود `price` كـ `outcome` بينما تم تعيين باقي الأعمدة كـ `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` يحدد أن جميع المتنبئين يجب تحويلهم إلى مجموعة من الأعداد الصحيحة تبدأ من الرقم 0.

نحن متأكدون أنك قد تكون لديك أفكار مثل: "هذا رائع جدًا!! ولكن ماذا لو كنت بحاجة إلى التأكد من أن الوصفات تقوم بالضبط بما أتوقعه؟ 🤔"

هذه فكرة رائعة! كما ترى، بمجرد تعريف الوصفة، يمكنك تقدير المعلمات المطلوبة لمعالجة البيانات فعليًا، ثم استخراج البيانات المعالجة. عادةً لا تحتاج إلى القيام بذلك عند استخدام Tidymodels (سنرى الطريقة المعتادة بعد قليل -> `workflows`) ولكن يمكن أن يكون ذلك مفيدًا عندما تريد إجراء نوع من التحقق للتأكد من أن الوصفات تعمل كما تتوقع.

لهذا، ستحتاج إلى فعلين إضافيين: `prep()` و `bake()` وكما هو الحال دائمًا، أصدقاؤنا الصغار في R من [Allison Horst](https://github.com/allisonhorst/stats-illustrations) يساعدونك في فهم هذا بشكل أفضل!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): يقوم بتقدير المعايير المطلوبة من مجموعة تدريب يمكن تطبيقها لاحقًا على مجموعات بيانات أخرى. على سبيل المثال، بالنسبة لعمود متنبئ معين، ما هي الملاحظة التي سيتم تخصيصها للعدد الصحيح 0 أو 1 أو 2 وما إلى ذلك.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): يأخذ وصفة تم تجهيزها مسبقًا ويطبق العمليات على أي مجموعة بيانات.

وبناءً على ذلك، دعونا نقوم بتجهيز وخبز وصفاتنا للتأكد فعلاً من أنه في الخلفية، سيتم ترميز أعمدة المتنبئ أولاً قبل أن يتم ضبط النموذج.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

رائع! 🥳 البيانات المعالجة `baked_pumpkins` تحتوي على جميع المتغيرات المستقلة مشفرة، مما يؤكد أن خطوات المعالجة المسبقة التي تم تعريفها كالوصفة تعمل كما هو متوقع. قد يكون من الصعب عليك قراءة البيانات الآن، لكنها أصبحت أكثر وضوحًا بالنسبة لـ Tidymodels! خذ بعض الوقت لتكتشف أي ملاحظة تم ربطها برقم صحيح.

من الجدير بالذكر أيضًا أن `baked_pumpkins` هو إطار بيانات يمكننا إجراء العمليات الحسابية عليه.

على سبيل المثال، دعنا نحاول العثور على ارتباط جيد بين نقطتين من بياناتك لبناء نموذج تنبؤي جيد. سنستخدم الدالة `cor()` للقيام بذلك. اكتب `?cor()` لمعرفة المزيد عن هذه الدالة.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


كما يتضح، هناك ارتباط ضعيف بين المدينة والسعر. ومع ذلك، هناك ارتباط أفضل قليلاً بين الحزمة وسعرها. هذا منطقي، أليس كذلك؟ عادةً، كلما كانت صندوق المنتجات أكبر، كان السعر أعلى.

وبما أننا نتحدث عن ذلك، دعونا نحاول أيضًا تصور مصفوفة الارتباط لجميع الأعمدة باستخدام حزمة `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 أفضل بكثير.

سؤال جيد يمكن طرحه الآن على هذه البيانات هو: '`ما السعر الذي يمكنني توقعه لحزمة معينة من القرع؟`' لنبدأ مباشرة!

> ملاحظة: عندما تقوم **`bake()`** بوصفة التحضير **`pumpkins_prep`** مع **`new_data = NULL`**، فإنك تستخرج بيانات التدريب المعالجة (أي المشفرة). إذا كان لديك مجموعة بيانات أخرى مثل مجموعة اختبار وترغب في رؤية كيف ستقوم الوصفة بمعالجتها مسبقًا، يمكنك ببساطة خبز **`pumpkins_prep`** مع **`new_data = test_set`**

## 4. بناء نموذج الانحدار الخطي

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>رسم توضيحي بواسطة داساني ماديبالي</figcaption>


<!--![رسم توضيحي بواسطة داساني ماديبالي](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ar.png){width="800"}-->


الآن بعد أن قمنا ببناء وصفة وتأكدنا فعليًا من أن البيانات ستتم معالجتها مسبقًا بشكل مناسب، دعونا الآن نبني نموذج انحدار للإجابة على السؤال: `ما هو السعر المتوقع لحزمة معينة من القرع؟`

#### تدريب نموذج انحدار خطي باستخدام مجموعة التدريب

كما قد تكون قد استنتجت بالفعل، فإن العمود *price* هو المتغير `النتيجة` بينما العمود *package* هو المتغير `المتنبئ`.

للقيام بذلك، سنقوم أولاً بتقسيم البيانات بحيث يتم تخصيص 80% لمجموعة التدريب و20% لمجموعة الاختبار، ثم نحدد وصفة تقوم بترميز عمود المتنبئ إلى مجموعة من الأعداد الصحيحة، ثم نبني مواصفات النموذج. لن نقوم بتحضير أو خبز الوصفة لأننا نعلم بالفعل أنها ستقوم بمعالجة البيانات كما هو متوقع.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

عمل رائع! الآن بعد أن أصبح لدينا وصفة ومواصفات نموذج، نحتاج إلى إيجاد طريقة لدمجهما معًا في كائن يقوم أولاً بمعالجة البيانات مسبقًا (prep+bake خلف الكواليس)، ثم تدريب النموذج على البيانات المعالجة مسبقًا، وأيضًا يتيح إمكانية القيام بأنشطة ما بعد المعالجة. ما رأيك في ذلك لراحة بالك! 🤩

في Tidymodels، يُطلق على هذا الكائن المريح اسم [`workflow`](https://workflows.tidymodels.org/) وهو يحتفظ بمكونات النمذجة الخاصة بك بكل سهولة! هذا ما نسميه *pipelines* في *Python*.

لذا، دعونا نجمع كل شيء في workflow! 📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

بالمثل، يمكن إعداد/تدريب سير العمل بنفس الطريقة التي يمكن بها تدريب النموذج.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

من خلال نتائج النموذج، يمكننا رؤية المعاملات التي تم تعلمها أثناء التدريب. هذه المعاملات تمثل معاملات خط التناسب الأفضل الذي يحقق أقل خطأ إجمالي بين المتغير الفعلي والمتغير المتوقع.

#### تقييم أداء النموذج باستخدام مجموعة الاختبار

حان الوقت لمعرفة كيف كان أداء النموذج 📏! كيف نقوم بذلك؟

الآن بعد أن قمنا بتدريب النموذج، يمكننا استخدامه لتقديم توقعات لمجموعة الاختبار باستخدام `parsnip::predict()`. بعد ذلك، يمكننا مقارنة هذه التوقعات بالقيم الفعلية للتصنيفات لتقييم مدى جودة (أو عدم جودة!) عمل النموذج.

لنبدأ بعمل التوقعات لمجموعة الاختبار ثم نربط الأعمدة بمجموعة الاختبار.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

نعم، لقد قمت بتدريب نموذج واستخدمته لإجراء التنبؤات! 🔮 هل هو جيد؟ دعنا نقيم أداء النموذج!

في Tidymodels، نقوم بذلك باستخدام `yardstick::metrics()`! بالنسبة للانحدار الخطي، دعنا نركز على المقاييس التالية:

-   `Root Mean Square Error (RMSE)`: الجذر التربيعي لـ [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). هذا يعطي مقياسًا مطلقًا بنفس وحدة التسمية (في هذه الحالة، سعر اليقطين). كلما كان القيمة أصغر، كان النموذج أفضل (بمعنى مبسط، يمثل متوسط السعر الذي تكون فيه التنبؤات خاطئة!).

-   `Coefficient of Determination (المعروف عادةً بـ R-squared أو R2)`: مقياس نسبي حيث كلما كانت القيمة أعلى، كان توافق النموذج أفضل. في جوهره، يمثل هذا المقياس مقدار التباين بين القيم المتوقعة والقيم الفعلية التي يستطيع النموذج تفسيرها.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

هناك تدهور في أداء النموذج. دعونا نرى ما إذا كان بإمكاننا الحصول على مؤشر أفضل من خلال تصور مخطط التبعثر بين الحزمة والسعر، ثم استخدام التوقعات التي تم إنشاؤها لرسم خط الاتجاه الأفضل.

هذا يعني أننا سنحتاج إلى تجهيز وإعداد مجموعة الاختبار من أجل ترميز عمود الحزمة، ثم ربط ذلك بالتوقعات التي أنشأها نموذجنا.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


رائع! كما ترى، نموذج الانحدار الخطي لا يعمم بشكل جيد العلاقة بين العبوة وسعرها المقابل.

🎃 تهانينا، لقد قمت للتو بإنشاء نموذج يمكنه المساعدة في التنبؤ بأسعار بعض أنواع القرع. ستكون مزرعة القرع الخاصة بك رائعة في موسم الأعياد. ولكن ربما يمكنك إنشاء نموذج أفضل!

## 5. بناء نموذج انحدار متعدد الحدود

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>رسم توضيحي من إعداد داساني ماديبالي</figcaption>


<!--![رسم توضيحي من إعداد داساني ماديبالي](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.ar.png){width="800"}-->


أحيانًا قد لا تكون بياناتنا ذات علاقة خطية، ولكننا نرغب مع ذلك في التنبؤ بالنتائج. يمكن أن يساعدنا الانحدار متعدد الحدود (Polynomial Regression) في إجراء التنبؤات للعلاقات الأكثر تعقيدًا وغير الخطية.

خذ على سبيل المثال العلاقة بين العبوة والسعر في مجموعة بيانات القرع الخاصة بنا. في بعض الأحيان، تكون هناك علاقة خطية بين المتغيرات - كلما زاد حجم القرع (من حيث الحجم)، زاد السعر - ولكن في أحيان أخرى، لا يمكن تمثيل هذه العلاقات كطائرة أو خط مستقيم.

> ✅ إليك [بعض الأمثلة الأخرى](https://online.stat.psu.edu/stat501/lesson/9/9.8) على بيانات يمكن استخدام الانحدار متعدد الحدود معها.
>
> ألقِ نظرة أخرى على العلاقة بين النوع والسعر في الرسم البياني السابق. هل يبدو أن هذا الرسم النقطي يجب تحليله بالضرورة باستخدام خط مستقيم؟ ربما لا. في هذه الحالة، يمكنك تجربة الانحدار متعدد الحدود.
>
> ✅ الحدوديات (Polynomials) هي تعبيرات رياضية قد تتكون من متغير واحد أو أكثر ومعاملات.

#### تدريب نموذج انحدار متعدد الحدود باستخدام مجموعة التدريب

يقوم الانحدار متعدد الحدود بإنشاء *خط منحني* ليتناسب بشكل أفضل مع البيانات غير الخطية.

دعنا نرى ما إذا كان النموذج متعدد الحدود سيؤدي بشكل أفضل في إجراء التنبؤات. سنتبع إجراءً مشابهًا لما قمنا به سابقًا:

- إنشاء وصفة تحدد خطوات المعالجة المسبقة التي يجب تنفيذها على بياناتنا لتجهيزها للنمذجة، مثل: ترميز المتنبئات وحساب الحدوديات من الدرجة *n*.

- بناء مواصفات النموذج.

- دمج الوصفة ومواصفات النموذج في سير عمل.

- إنشاء نموذج من خلال ملاءمة سير العمل.

- تقييم أداء النموذج على بيانات الاختبار.

لنبدأ مباشرة!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### تقييم أداء النموذج

👏👏 لقد قمت ببناء نموذج متعدد الحدود، دعنا نقوم بعمل تنبؤات على مجموعة الاختبار!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

ووهو، دعونا نقيم أداء النموذج على مجموعة_الاختبار باستخدام `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 أداء أفضل بكثير.

انخفض `rmse` من حوالي 7 إلى حوالي 3، مما يشير إلى تقليل الخطأ بين السعر الفعلي والسعر المتوقع. يمكنك *بشكل عام* تفسير هذا على أنه يعني أن التوقعات الخاطئة تكون في المتوسط خاطئة بحوالي 3 دولارات. ارتفع `rsq` من حوالي 0.4 إلى 0.8.

تشير جميع هذه المقاييس إلى أن النموذج متعدد الحدود يقدم أداءً أفضل بكثير مقارنة بالنموذج الخطي. عمل رائع!

دعونا نرى إذا كان بإمكاننا تصور ذلك!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


يمكنك رؤية خط منحني يتناسب بشكل أفضل مع بياناتك! 🤩

يمكنك جعله أكثر سلاسة عن طريق تمرير صيغة متعددة الحدود إلى `geom_smooth` بهذا الشكل:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

مثل منحنى سلس! 🤩

إليك كيفية إنشاء توقع جديد:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


توقعات `polynomial model` تبدو منطقية، بالنظر إلى الرسوم البيانية المبعثرة لـ `price` و `package`! وإذا كان هذا النموذج أفضل من النموذج السابق، بالنظر إلى نفس البيانات، ستحتاج إلى تخصيص ميزانية لهذه القرع الأغلى ثمناً!

🏆 أحسنت! لقد أنشأت نموذجين للتنبؤ في درس واحد. في القسم الأخير عن الانحدار، ستتعلم عن الانحدار اللوجستي لتحديد الفئات.

## **🚀التحدي**

اختبر عدة متغيرات مختلفة في هذا الدفتر لترى كيف تتوافق العلاقة بين الارتباط ودقة النموذج.

## [**اختبار ما بعد المحاضرة**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **المراجعة والدراسة الذاتية**

في هذا الدرس تعلمنا عن الانحدار الخطي. هناك أنواع أخرى مهمة من الانحدار. اقرأ عن تقنيات Stepwise، Ridge، Lasso و Elasticnet. دورة جيدة لدراسة المزيد هي [دورة التعلم الإحصائي من جامعة ستانفورد](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

إذا كنت ترغب في تعلم المزيد عن كيفية استخدام إطار العمل الرائع Tidymodels، يرجى الاطلاع على الموارد التالية:

-   موقع Tidymodels: [ابدأ مع Tidymodels](https://www.tidymodels.org/start/)

-   ماكس كون وجوليا سيلج، [*النمذجة المنظمة باستخدام R*](https://www.tmwr.org/)*.*

###### **شكر خاص إلى:**

[أليسون هورست](https://twitter.com/allison_horst?lang=en) على إنشاء الرسوم التوضيحية الرائعة التي تجعل R أكثر ترحيباً وجاذبية. يمكنك العثور على المزيد من الرسوم التوضيحية في [معرضها](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
